In this script you can analyse a single simulation in critical exporations.

It can be split in two parts:

-The first examines the behaviour of various quantities (scalar field, lapse, Ricci scalar etc) at r=0. To have good results it is suggested that you run with an infalling outer boundary rmax, but this analyses also works as is, with fixed rmax.

-The second part analyses these quantities both in time and space. They are ploted as functions of the radial domain r, for different timesteps. As is, the analysis assumes an infalling rmax, and so it loads the radial grid r for every timestep. If you run with a fixed timestep, you have to modify that specific part. 

In [ ]:
# load packages
using HDF5
using LaTeXStrings
using Plots; pythonplot()
using DelimitedFiles
using Interpolations
using SpheriCo

In [ ]:
# give the directory where the data from all the runs are saved
base = "../examples/classical_runs/"
par = "a0.0449_b5.0_c1.0_rmax11.76_tmax11.75_cfl0.125_sigma0.02_infalling_rmax_true_rand_false_overMp2_25.132741228718345_damp0"

dir = base*par
# convention of 1/M_Planck^2; set manually
oMp2 = 25.132741228718345

In [ ]:
# set resolution manually (number of points in radial domain)
D = 5
Nr = (2^D)*128 + 3

# list all available iterations (and corresponding files)
(its, all_filenames) = list_h5_files(dir*"/data_$(Nr)", prefix="r0data_")
println(length(its))

Load the data at r=0, for the lapse, scalar field, Ricci scalar, Hamiltonian and momentum constraint violation, as well as the difference between the metric functions A,B and the extrinsic curvature components KA, KB.

In [ ]:
# the classical state vector is:
#v_classic_labels = ["Φ", "Π", "Ψ", "A", "B", "DB", "Utld", "K", "KB", "λ", "α", "Dα", "Θ", "Zr", "f", "g", "U", "V"]

# initiate lists to save the data at r=0, for all the timesteps for which r0data type is saved
# lapse
α_r0_t         = zeros(length(its));
# scalar field
Φ_r0_t         = zeros(length(its));
# Ricci scalar
R_r0_t         = zeros(length(its));
# Hamiltonian constraint violation
H_r0_t         = zeros(length(its));
# momentum constraint violation
P_r0_t         = zeros(length(its));
# difference of metric functions A,B
AminusB_r0_t   = zeros(length(its));
# difference of extrinsic curvature components KA, KB
KAminusKB_r0_t = zeros(length(its));

# list of timesteps
t_list = zeros(length(its));

# pass through all the saved timesteps and save the corresponding data
for i in 1:length(its)
    it = its[i]
    it_str  = lpad(it, 4, "0")

    # load the classical state vector
    v = h5read(dir*"/data_$(Nr)/r0data_$(it_str).h5","v")
    # load the radial grid 
    r = h5read(dir*"/data_$(Nr)/r0data_$(it_str).h5","r")
    # calculate grid spacing
    dr = r[4] - r[3]
    # timestep
    t = h5readattr(dir*"/data_$(Nr)/r0data_$(it_str).h5", "/")["time"]
    # calculate Hamitlonian (H) and momentum (P) constraint violation
    H, P = SpheriCo.classical.constraints(v, dr, oMp2);
    # calculate the Ricci scalar
    R  = SpheriCo.Ricci_scalar(v, r, oMp2);

    # store the data in the corrsponding lists
    α_r0_t[i] = v[3,11]
    Φ_r0_t[i] = v[3,1]
    R_r0_t[i] = R[3]
    H_r0_t[i] = H[3]
    P_r0_t[i] = P[3]
    AminusB_r0_t[i] = v[3,4]-v[3,5] # A-B
    KAminusKB_r0_t[i] = v[3,8]-3*v[3,9] #K - 3*KB

    # store timesteps
    t_list[i] = t
end

In [ ]:
# find proper time at r=0
dt = t_list[2]-t_list[1];
proper_time_list = zeros(length(t_list));
for i in 2:length(proper_time_list)
    proper_time_list[i] = proper_time_list[i-1] + α_r0_t[i]*dt
end

We want to find similarity time, which is useful in the plots to see better the behaviour at r=0. For this, we need two pairs of consequtive zero crossings. Accumulation time is roughly when the scalar field starts to oscilate fast at r=0.

In [ ]:
# find accumulation time; we need two pairs of consequtive zero crossings
test_sign = sign(Φ_r0_t[1])
zero_crossings = []
for i in 1:length(proper_time_list)
    if test_sign != sign(Φ_r0_t[i])
        append!(zero_crossings, proper_time_list[i])
        test_sign = sign(Φ_r0_t[i])
    end
end
println(zero_crossings)

Plot the scalar field at r=0 against coordinate time. Near criticallity the scalar field is expected to oscillate between +-0.6, regardless of the initial data. This is sometimes refered to as universality of solutions in spherically symmetric gravitational critical phenomena.

In [ ]:
ti = 1 # timestep in t_list when you start ploting
tf = length(proper_time_list) # timestep in t_list when you stop ploting
plot(t_list[ti:tf], Φ_r0_t[ti:tf], title = "r=0",  label="Φ(t)", linewidth=3, legend=:topleft)
plot!(t_list[ti:tf], -0.6*ones(tf), linewidth=1, color="black", label="")
plot!(t_list[ti:tf], 0.6*ones(tf), linewidth=1, color="black", label="")

Plot the lapse at r=0, against coordinate time:

In [ ]:
ti = 1
tf = length(proper_time_list)
plot(t_list[ti:tf], α_r0_t[ti:tf], title = "r=0",  label="α(t)", linewidth=3, legend=:topleft)

Plot the scalar field at r=0, against the proper time at r=0:

In [ ]:
ti = 1
tf = length(proper_time_list)
plot(proper_time_list[ti:tf], Φ_r0_t[ti:tf], title = "r=0",  label=L"Φ(\tau,r=0)", linewidth=3)

Find the similarity time. This is usefule to see better the behaviour of the scalar field (and other quantities) at r=0 near criticality. It requires finding first the accumulation time, which needs four consequtive zero crossings of the scalar field (oscilations passing through the value zero).

In [ ]:
# print the consequtive zero crossings
println("zero crossings: ", zero_crossings)
# choose the pairs of consequtive zero crossings used to calculate the accumulation time
un = zero_crossings[1]
unplus1 = zero_crossings[2]
um = zero_crossings[3]
umplus1 = zero_crossings[4]
# evaluate the formula for accumulation time
accum_time = (un*umplus1 - unplus1*um)/(un - unplus1 - um + umplus1)
# print the accumulation time
println("accumulation time: ", accum_time)
# find the similarity time T = -ln(u*-u), where u is proper time at r=0 and u* accumulationtime
similarity_time = []
i = 1
while proper_time_list[i] < accum_time
    T = -log(accum_time - proper_time_list[i])
    append!(similarity_time, T)
    i += 1
end
#println(similarity_time)

In [ ]:
ti = 1
tf = length(similarity_time)

plot(similarity_time[ti:tf], Φ_r0_t[ti:tf], title = "r=0",  label=L"\Phi(\tau)", linewidth=3)
plot!([0.6], seriestype="hline", label="0.6", linewidth=3, color = "black")
p1 = plot!([-0.6], seriestype="hline", label="-0.6", linewidth=3, color = "black", xaxis=nothing, frame=true)

p2 = plot(similarity_time[ti:tf], α_r0_t[ti:tf], label=L"\alpha(\tau)", linewidth=3, xaxis=nothing, frame=true)

p3 = plot(similarity_time[ti:tf], abs.(R_r0_t[ti:tf]), label=L"|R(\tau)|", linewidth=3, xaxis=nothing, frame=true)

p4 = plot(similarity_time[ti:tf], abs.(H_r0_t[ti:tf]), label=L"|H(\tau)|", linewidth=3, xaxis=nothing, frame=true)

p5 = plot(similarity_time[ti:tf], abs.(P_r0_t[ti:tf]), label=L"|P(\tau)|", linewidth=3, xaxis=nothing, frame=true)

p6 = plot(similarity_time[ti:tf], abs.(AminusB_r0_t[ti:tf]), label=L"|A(\tau)-B(\tau)|", linewidth=3, frame=true,
xlabel="similarity time")

p7 = plot(similarity_time[ti:tf], abs.(KAminusKB_r0_t[ti:tf]), label=L"|K_A(\tau)-K_B(\tau)|", linewidth=3, frame=true,
xlabel="similarity time")

plt = plot(p1, p3, p4, p5, p6, p7, 
    layout = grid(3, 2),
    wsize = (800,600))

# uncomment if you want to save the figure, and modify appropriately
#savefig(plt, "./critical/"*par*"_D$(D).pdf")

Plot the proper time at r=0, against coordinate time:

In [ ]:
plot(t_list[1:end], proper_time_list[1:end], title = "r=0",  label="proper_time_r0(t)",
    linewidth=3, frame=true, legend=:topleft, xlabel="time", ylabel="proper time @ r=0")

Below you can create animations that show the evolution e.g. of the scalar field (or other quantities) in the whole radial domain, for the different timesteps. For this you need the data type "data" and not "r0data". If you run with an infalling rmax, then "data" also saves the radial grid at each timestep, otherwise not.

List all the "data" files:

In [ ]:
# list all available iterations (and corresponding files)
(its2, all_filenames2) = list_h5_files(dir*"/data_$(Nr)", prefix="data_")
println(length(its2))

In [ ]:
# the classical state vector is:["Φ", "Π", "Ψ", "A", "B", "DB", "Utld", "K", "KB", "λ", "α", "Dα", "Θ", "Zr", "f", "g", "U", "V"]
v_classic_labels = ["Φ", "Π", "Ψ", "A", "B", "DB", "Utld", "K", "KB", "λ", "α", "Dα", "Θ", "Zr", "f", "g", "U", "V"]

# choose the index that corresponds to the function you want to plot, in the state vector
fi = 1 # 1 is the scalar field Φ

# set the step for the animation below (if you have many steps saved it takes a while). Suggestion: use powers of 2
step = 2^0
anim = @animate for i ∈ 1:step:length(its2)
    it = its2[i]
    it_str  = lpad(it, 4, "0")
    f  =  h5read(dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,fi]
    t  =  h5readattr(dir*"/data_$(Nr)/data_$(it_str).h5", "/")["time"]
    # if you run with fixed rmax, comment the line below
    r  =  h5read(dir*"/data_$(Nr)/data_$(it_str).h5","r")

    plot(r[3:end], f[3:end], title = "t=$(t)", label=v_classic_labels[fi], linewidth=3)
end

# creates a movies directory within the tools directory, and saves the gif
if ispath("./movies/")==false
    mkpath("./movies/")
end

gif(anim, "./movies/"*v_classic_labels[fi]*".gif", fps = 5) # fps is frames per second; the larger it is the faster the gif

In [ ]:
# set the step for the animation below (if you have many steps saved it takes a while). Suggestion: use powers of 2
step = 2^2

anim = @animate for i ∈ 1:step:length(its2)
    it = its2[i]#[i] for when they are saved on common timesteps # otherwise [1+i]
    it_str  = lpad(it, 4, "0")

    # load the classical state vector
    v = h5read(dir*"/data_$(Nr)/data_$(it_str).h5","v")
    # load the radial grid; if you run with fixed rmax, comment the two lines below out
    r = h5read(dir*"/data_$(Nr)/data_$(it_str).h5","r")
    # calculate grid spacing
    dr = r[4] - r[3]
    # timestep
    t = h5readattr(dir*"/data_$(Nr)/data_$(it_str).h5", "/")["time"]
    # calculate Hamitlonian (H) and momentum (P) constraint violation
    H, P = SpheriCo.classical.constraints(v, dr, oMp2);
    # calculate the Ricci scalar
    R  = SpheriCo.Ricci_scalar(v, r, oMp2);
    # lapse
    α = v[:,11]
    # scalar
    Φ = v[:,1]
    
    # ri and rf are start and finish of the radial grid in the plots
    ri = 3
    # change if you want to not plot rmax
    rf = length(r) -100 #  removing points near rmax, because the violations there are bigger but not relevant
    
    # Plots
    p1 = plot(r[ri:rf], Φ[ri:rf], title = "t=$(t)",  label=L"\Phi(r)", linewidth=3, frame=true, xaxis=nothing)
    p2 = plot(r[ri:rf], abs.(R[ri:rf]), label=L"|R(r)|", linewidth=3, frame=true, xaxis=nothing)
    p3 = plot(r[ri:rf], α[ri:rf], label=L"\alpha(r)", linewidth=3, frame=true, xaxis=nothing)
    p4 = plot(r[ri:rf], abs.(H[ri:rf]),  label=L"|H(r)|", linewidth=3, frame=true, xlabel="r")
    p5 = plot(r[ri:rf], abs.(P[ri:rf]), label=L"|P(r)|", linewidth=3, frame=true, xlabel="r")
    
    plot(p1, p2, p3, p4, p5, layout=(3,2), wsize = (800,600))
end

gif(anim, "./movies/Phi_lapse_R_H_P.gif", fps = 5)